In [1]:
import torch
_ = torch.tensor([0.2126, 0.7152, 0.0722], names=['c'])

/var/folders/wm/znp8c4pd0r72qm3gq7rpvwn40000gn/T/ipykernel_71217/1646494204.py:2: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/c10/core/TensorImpl.h:1938.)
  _ = torch.tensor([0.2126, 0.7152, 0.0722], names=['c'])


In [2]:
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [3]:
img_t

tensor([[[-2.8156e+00,  5.9524e-01,  1.7479e+00,  4.4990e-01,  3.7172e-01],
         [ 9.8252e-01,  1.8241e-01, -1.3008e+00, -3.0076e-01,  8.8671e-01],
         [-1.0118e+00, -1.3985e+00, -2.2884e+00, -3.0381e-01, -1.7115e-01],
         [-1.3628e-01,  1.4907e+00, -9.1903e-01,  5.8590e-01,  2.6922e-01],
         [-6.4469e-01, -1.7688e+00, -2.4582e-01,  7.3407e-01,  5.4681e-01]],

        [[-4.9586e-01, -1.5140e+00,  1.0339e+00,  1.3441e+00,  1.4587e+00],
         [-1.8406e-01,  9.4507e-01,  9.8145e-01,  4.8483e-01, -2.6738e-01],
         [ 8.3782e-01, -1.7631e+00,  9.7122e-01,  7.8556e-01,  1.8709e+00],
         [ 1.4523e+00, -5.6635e-01,  1.1977e+00,  5.5426e-01, -1.2893e-01],
         [ 5.5783e-01, -1.3639e+00,  6.2563e-01, -2.9700e-01, -8.2193e-01]],

        [[-7.4259e-01,  4.3760e-01, -1.5437e+00,  2.0606e+00,  5.9799e-01],
         [-9.2020e-01, -1.5861e+00,  2.2358e-01, -6.1514e-01, -4.8923e-01],
         [-8.0538e-01,  7.0999e-02,  8.2934e-01,  1.5806e+00, -1.0218e-01],
        

In [4]:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]

In [5]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [6]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [7]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [8]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [9]:
img_named =  img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [10]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [11]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [12]:
try:
    gray_named = (img_named[..., :3] * weights_named).sum('channels')
except Exception as e:
    print(e)

Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.


In [13]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))